## Urban Heat Island (UHI) effect reduction 
### Written by: Amy Tran

## Scenario

## Objective

# Datasets
trees-with-species-and-dimensions-urban-forest<br>
laneways-with-greening-potential<br>
microclimate-sensor-readings<br>

## Codes

#### Load data 

In [159]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import plotly.express as px
import geopy.distance

#pip install plotly==5.8.0
#pip install geopy

In [206]:
#Sample query
#import requests
#session = requests.Session()
#base = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'
#url = 'microclimate-sensor-readings'
#filters = f'records?limit={10}&offset={9990}&timezone=UTC'
#filters = f'records?limit={10}&offset={0}&timezone=UTC'
#target_url = f'{base}{url}/{filters}'
#result = session.get(target_url+f'&apikey={API_KEY}')
#result

<Response [400]>

In [160]:
#Create function to calculate distance between 2 coordinates
import geopy.distance
def distance(lat1, lon1, lat2, lon2):
    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)
    km = geopy.distance.geodesic(coords_1, coords_2).km
    return (km)

#Create function to make iterative calls 
def get_data(base, url, size = 0):
    #Extract 1st row as default result 
    #default_filters = filters = f'records?limit={1}&offset={0}&timezone=UTC'
    #default_url = f'{base}{url}/{default_filters}'
    #default_result = session.get(default_url)
    #default_result_json = default_result_json()
    #default_max_results = default_result_json['total_count']

    #Extract target result (full data)
    target_filters = f'records?limit={10}&offset={size}&timezone=UTC'
    target_url = f'{base}{url}/{target_filters}'
    result = session.get(target_url+f'&apikey={API_KEY}')
    status_code = result.status_code
    if status_code == 200:
        result_json = result.json()
        max_results = result_json['total_count']
        links = result_json['links']
        records = result_json['records']
        records_df = pd.json_normalize(records)
    
        #Update column labels
        records_df.drop(columns=['links'],inplace=True)
        column_names = records_df.columns.values.tolist()

        #Replace geolocation.lat & geolocation.lon
        column_names = ['_'.join((a.split(".")[-2:])) if a.split('.')[-2]=='geolocation' else a for a in column_names]
        column_names = [i.split('.')[-1] for i in column_names]
        records_df.columns = column_names
    
        next_url = None
             
        #Obtain next url
        if records_df.shape[0] != max_results:
            for l in links:
                if l['rel'] == 'next':
                    next_url = l['href']
    
        return[records_df, next_url, column_names, max_results, status_code]
    else: return[None, None, None, None, status_code]

In [162]:
#Extract laneways-with-greening-potential data 
session = requests.Session()
base = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'
url = 'laneways-with-greening-potential'

target_url = f'{base}{url}/exports/json'
result = session.get(target_url)
result_json = result.json()
data = pd.json_normalize(result_json)
laneways = data.copy()
#Rename Longitude and Latitude columns
laneways = laneways.rename(columns = {'geo_point_2d.lon' : 'lon', 'geo_point_2d.lat' : 'lat'})
print(laneways.shape)

(246, 26)

In [163]:
#Extract urban forest data 
session = requests.Session()
base = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'
url = 'trees-with-species-and-dimensions-urban-forest'

target_url = f'{base}{url}/exports/json'
result = session.get(target_url)
result_json = result.json()
data = pd.json_normalize(result_json)
uf = data.copy()
#Rename Longitude and Latitude columns
uf = uf.rename(columns = {'coordinatelocation.lon' : 'lon', 'coordinatelocation.lat' : 'lat'})
print(uf.shape)

(76928, 22)

In [171]:
#Extract energy consumption projection data 
session = requests.Session()
base = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'
url = 'block-level-energy-consumption-modelled-on-building-attributes-2026-projection-r'

target_url = f'{base}{url}/exports/json'
result = session.get(target_url)
result_json = result.json()
data = pd.json_normalize(result_json)
energy = data.copy()
#Rename Longitude and Latitude columns
energy = energy.rename(columns = {'geo_point_2d.lon' : 'lon', 'geo_point_2d.lat' : 'lat'})
print(energy.shape)

(628, 6)


In [174]:
#Count number of trees in each greening location and its surrounding within the radius set
#Set radius
radius = 0.1 #KM
location_count = len(laneways)

#Filter Urban Forest dataset to include only trees within the bounding box
#Create bounding box
#Find Min and Max latitude and longitude of greening location
max_lon = laneways['lon'].max()
max_lat = laneways['lat'].max()
min_lon = laneways['lon'].min()
min_lat = laneways['lat'].min()

#Find coordinate of bounding box 
b = [45, 135, 225, 315]
d = radius

#North East (NE) coordinate of bounding box
origin = geopy.Point(max_lat, max_lon)
destination = geopy.distance.geodesic(kilometers=d).destination(origin, b[0])
NE_lat, NE_lon = destination.latitude, destination.longitude

#South East (SE) coordinate of bounding box
origin = geopy.Point(min_lat, max_lon)
destination = geopy.distance.geodesic(kilometers=d).destination(origin, b[1])
SE_lat, SE_lon = destination.latitude, destination.longitude

#South West (SW) coordinate of bounding box
origin = geopy.Point(min_lat, min_lon)
destination = geopy.distance.geodesic(kilometers=d).destination(origin, b[2])
SW_lat, SW_lon = destination.latitude, destination.longitude

#North West (NW) coordinate of bounding box
origin = geopy.Point(max_lat, min_lon)
destination = geopy.distance.geodesic(kilometers=d).destination(origin, b[3])
NW_lat, NW_lon = destination.latitude, destination.longitude

#Latitude boundaries
max_lat_bound = max(NW_lat, NE_lat)
min_lat_bound = min(SW_lat, SE_lat)
#Longitude boundaries
max_lon_bound = max(NE_lon, SE_lon)
min_lon_bound = min(NW_lon, SW_lon)

#Filter Urban Forest dataset to include only trees within the bounding box
uf_df = uf.loc[(uf['lat'].between(min_lat_bound, max_lat_bound)) & (uf['lon'].between(min_lon_bound, max_lon_bound))]
uf_df = uf_df.reset_index()

#Filter Energy Consumption Projection dataset to include only sites within the bounding box
energy_df = energy.loc[(energy['lat'].between(min_lat_bound, max_lat_bound)) & (energy['lon'].between(min_lon_bound, max_lon_bound))]
energy_df = energy_df.reset_index()

#Count number of trees and sum up projected energy consumption in each greening location and its surrounding within the radius set
for i in range(location_count):
#for i in range(2):
    #Longitude and Latitude of greening area
    green_lon = laneways.loc[i, 'lon']
    green_lat = laneways.loc[i, 'lat']
    #Count number of trees within set radius
    tree_count = 0
    for x in range(len(uf_df)):
        tree_lon = uf_df.loc[x, 'lon']
        tree_lat = uf_df.loc[x, 'lat']
        km = distance(tree_lat, tree_lon, green_lat , green_lon)
        if abs(km) < radius:
            tree_count = tree_count 
            tree_count = tree_count + 1
    laneways.loc[i, 'tree_count'] = tree_count
    #Total projected energy consumption within set radius
    total_energy = 0
    for k in range(len(energy_df)):
        site_lon = energy_df.loc[k, 'lon']
        site_lat = energy_df.loc[k, 'lat']
        site_total = energy_df.loc[k, 'total']
        site_km = distance(site_lat, site_lon, green_lat , green_lon)
        if abs(site_km) < radius:
            total_energy = total_energy
            total_energy = total_energy + site_total
    laneways.loc[i, 'total_energy'] = total_energy
    print('Location:', i + 1, 'out of', location_count, ', Latitude:', green_lat, ', Longitude:', green_lon, ', Number of trees:', tree_count, ', Total energy consumption projected:', total_energy)


Location: 1 out of 246 , Latitude: -37.81980050254932 , Longitude: 144.9623447558369 , Number of trees: 29 , Total energy consumption projected: 0.0
Location: 2 out of 246 , Latitude: -37.81097635318543 , Longitude: 144.97170834375964 , Number of trees: 41 , Total energy consumption projected: 35193.2513804
Location: 3 out of 246 , Latitude: -37.81111151578862 , Longitude: 144.97244046374837 , Number of trees: 35 , Total energy consumption projected: 14548.8747872
Location: 4 out of 246 , Latitude: -37.81161232134716 , Longitude: 144.97077449834737 , Number of trees: 54 , Total energy consumption projected: 14548.8747872
Location: 5 out of 246 , Latitude: -37.8124734926234 , Longitude: 144.9713761243581 , Number of trees: 86 , Total energy consumption projected: 19579.5808423
Location: 6 out of 246 , Latitude: -37.81304184241003 , Longitude: 144.97240833983707 , Number of trees: 69 , Total energy consumption projected: 41492.609660500006
Location: 7 out of 246 , Latitude: -37.814104550

In [201]:
#Heatmap to show tree distribution
fig = px.density_mapbox(laneways, lat = 'lat', lon = 'lon', z = 'total_energy',mapbox_style="stamen-terrain", opacity = 0.5, 
        title = 'Heatmap showing projected energy consumption across City of Melbourne', 
        width = 1500, height = 1000, zoom = 14)
fig